# Pre Run Checks
 - Change runtime to **python3** with **GPU** Accelerator
 - Change default indent to **4** instead of **2**
 - Use **Dark theme** with **line numbers enabled** for your _sanity_
 - Logged in with **college** ID which contains the _reorganised_ dataset in the **Google Drive**
 - Check the _**decompression cell**_ & _**training cell**_ for **path** validity to the folder containing the compressed dataset
 - Get something ready to do for **10 mins** like _drinking coffee_ or turning on _**kitty mode**_ in _tools -> preferences -> misc_ to pass the time while it decompresses the dataset
 - Now run it & enjoy your coffee (or whatever is your poison)

In [1]:
!pip install -U pip
!pip install tensorflow-gpu==2.0.0-rc1

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.2.3)
     |████████████████████████████████| 380.5MB 28kB/s 
     |████████████████████████████████| 4.3MB 24.2MB/s 
     |████████████████████████████████| 501kB 38.2MB/s 


In [2]:
import os
import pathlib
import time
import tensorflow as tf

from google.colab import drive
drive.mount('/content/gdrive')

tf.random.set_seed(13) # Just a random seed for tensorflow

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Decompress the compressed reorganized "Jester" dataset in Google Colab VM

start = time.time()
!cat /content/gdrive/My\ Drive/DashGC/Reorganized/Dataset.tar.gz.?? | tar -xz
stop = time.time()
print('Decompression took', round(((stop - start) / 60), 2), 'mins')

Decompression took 11.38 mins


In [4]:
# Use Pathlib to get the path root to the dataset

train_data_root_orig = "./Dataset/Train"
train_data_root = pathlib.Path(train_data_root_orig)
print(train_data_root)

val_data_root_orig = "./Dataset/Validation"
val_data_root = pathlib.Path(val_data_root_orig)
print(val_data_root)

Dataset/Train
Dataset/Validation


In [5]:
# Sorting path list by filename (aka frame id) [first] & directory name (aka video id) [second sort key]

start = time.time()

val_image_paths = list(val_data_root.glob('*/*/*'))
# val_image_paths = [str(path) for path in val_image_paths]
val_image_paths = list(map(lambda x: str(x), val_image_paths))
val_image_paths.sort(key = lambda x: (int(x.split('/')[-1][:-5]), int(x.split('/')[-2])))
# val_image_paths = list(map(lambda s: pathlib.Path(s), val_image_paths))

train_image_paths = list(train_data_root.glob('*/*/*'))
# train_image_paths = [str(path) for path in train_image_paths]
train_image_paths = list(map(lambda x: str(x), train_image_paths))
train_image_paths.sort(key = lambda x: (int(x.split('/')[-1][:-5]), int(x.split('/')[-2])))
# train_image_paths = list(map(lambda s: pathlib.Path(s), train_image_paths))

stop = time.time()
print('Sorting took', round(((stop - start) / 60), 2), 'mins')

Sorting took 1.09 mins


In [6]:
print(type(train_image_paths[0]), train_image_paths[0])

<class 'str'> Dataset/Train/Doing other things/1/00003.jpg


In [0]:
# Indexing of classification class labels

label_names = sorted(item.name for item in train_data_root.glob('*/') if item.is_dir())
label_to_index = {name: index for index, name in enumerate(label_names)}

In [8]:
# Stores the label index for each video

start = time.time()

# train_image_labels = [label_to_index[pathlib.Path(path).parent.parent.name]
                    # for path in train_image_paths]

train_image_labels = list(map(lambda p: label_to_index[pathlib.Path(p).parent.parent.name], train_image_paths))

# val_image_labels = [label_to_index[pathlib.Path(path).parent.parent.name]
                    # for path in val_image_paths]

val_image_labels = list(map(lambda p: label_to_index[pathlib.Path(p).parent.parent.name], val_image_paths))

stop = time.time()
print('Labeling took', round(((stop - start) / 60), 2), 'mins') # 0.69 mins

Labeling took 0.67 mins


In [0]:
# Decode the video frames as int64 values of color & recast it as float32
# Resize the video frames into a squared size (100, 100, channels = 3)

#@tf.function
def preprocess_image(image):
    '''
    Decodes the binary form to normalized vector (aka tensor) form
    '''
    # image = tf.image.decode_image(image, channels=3)
    image = tf.image.decode_jpeg(image, channels=3) # imports RGB channels, jpeg doesn't have a 4th channel alpha for transparency like png
    image = tf.image.resize(image, [100, 100]) # original dimensions are (100, 176) (height, width)
    image /= 255.0  # normalize to [0,1] range using broadcast method
    return image

def load_and_preprocess_image(path):
    '''
    Open the file from path as binary
    '''
    image = tf.io.read_file(path)
    return preprocess_image(image)

def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

In [0]:
# Load the images as tensors (numpy-like multidimensional arrays)

train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
val_path_ds = tf.data.Dataset.from_tensor_slices(val_image_paths)
val_image_ds = val_path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
# Labeling

train_label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(train_image_labels, tf.int8))
val_label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(val_image_labels, tf.int8))

In [0]:
# Dataset Generator [WIP] [TODO]

train_image_label_ds = tf.data.Dataset.from_generator((pair for pair in zip(train_image_ds, train_label_ds)))
val_image_label_ds = tf.data.Dataset.from_generator((pair for pair in zip(val_image_ds, val_label_ds)))

In [0]:
# Dataset Generator [WIP] [TODO]

#train_image_label_ds = tf.data.Dataset.from_generator((pair for pair in zip(list(map(load_and_preprocess_image, train_image_paths)), train_label_ds)))
val_image_label_ds = tf.data.Dataset.from_generator((pair for pair in zip(list(map(load_and_preprocess_image, val_image_paths)), val_label_ds)))

ResourceExhaustedError: ignored

# Model 
The code following this block is to be customised for experimentation, testing & evaluation.

In [0]:
X_train = train_image_ds
X_val = val_image_ds

Y_train = train_label_ds
Y_val = val_label_ds

In [0]:
# [WIP] [TODO]

def cnn_block(X, filters, stage, block):
    """
    Implementation of the CNN block as defined

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the custom block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3, F4, F5 = filters

    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = tf.keras.layers.Conv2D(filters=F1, kernel_size=(3, 3), strides=(1, 1), padding='same', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = tf.keras.layers.Activation('relu')(X)

    X = tf.keras.layers.Conv2D(filters=F2, kernel_size=(3, 3), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = tf.keras.layers.Activation('relu')(X)

    X = tf.keras.layers.SpatialDropout2D(0.2)(X)
    X = tf.keras.layers.MaxPooling2D((2, 2), name='max_pool0')(X)

    # Second component of main path (≈2 lines)
    X = tf.keras.layers.Conv2D(filters=F3, kernel_size=(3, 3), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    X = tf.keras.layers.MaxPooling2D((2, 2), name='max_pool1')(X)

    X = tf.keras.layers.Conv2D(filters=F4, kernel_size=(3, 3), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2d')(X)
    X = tf.keras.layers.MaxPooling2D((2, 2), name='max_pool2')(X)

    X = tf.keras.layers.Conv2D(filters=F5, kernel_size=(3, 3), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2e')(X)
    X = tf.keras.layers.SpatialDropout2D(0.2)(X)
    X = tf.keras.layers.MaxPooling2D((2, 2), name='max_pool3')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)

    return X

In [0]:
# [WIP] [TODO]
def LSTM_block(X, f, filters, stage, block):
    """
    Implementation of the ConvLSTM block as defined

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the custom block, tensor of shape (n_H, n_W, n_C)
    """
    pass

In [0]:
# Model Definition using Keras Functional API

def model(input_shape):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = tf.keras.Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    # X = ZeroPadding2D((3, 3))(X_input)

    X = cnn_block(X_input, (1024, 512, 512, 256, 128), 0, "input_cnn")

    X = LSTM_block(X, 3, (128, 128, 64, 64, 32), 1, "output_lstm")

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(1, activation='softmax', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = tf.keras.Model(inputs = X_input, outputs = X, name='test0')

    return model

In [0]:
# Instantiate the model

dummy = model(input_shape=(100, 100, 3))

# Compile the model
dummy.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy', 'categorical_accuracy']
              )

In [0]:
# Training the model on the Dataset

history = dummy.fit(x=X_train,
                    y=Y_train,
                    batch_size=70,
                    epochs=20,
                    callbacks=tf.keras.callbacks.ModelCheckpoint(filepath='/content/gdrive/My Drive/DashGC/Reorganized/saved_models',
                                                                 monitor='val_loss',
                                                                 save_best_only=True,
                                                                 save_weights_only=False # Since we don't have a storage problem yet
                                                                 ),
                    validation_data=(X_val, Y_val)
                    )